In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

21/08/15 10:45:08 WARN Utils: Your hostname, DESKTOP-09QASF3 resolves to a loopback address: 127.0.1.1; using 172.20.197.123 instead (on interface eth0)
21/08/15 10:45:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/15 10:45:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.csv("data.csv", inferSchema=True, header=True)

In [3]:
from pyspark.ml.feature import VectorAssembler, MaxAbsScaler
from data import cols, labels

assembler1 = VectorAssembler(inputCols=cols, outputCol="features")
assembler2 = VectorAssembler(inputCols=labels, outputCol="label")
scaler = MaxAbsScaler(inputCol="features", outputCol="featuresScaled")

In [4]:
from pyspark.ml.pipeline import Pipeline
pipeline = Pipeline(stages=[assembler1, assembler2, scaler])

In [5]:
model = pipeline.fit(df)
data = model.transform(df)

In [6]:
train_set, test_set = data.randomSplit([.8, .2], 1234)

In [7]:
from knn import KNNClassifier
from pyspark.sql import DataFrame
classifier = KNNClassifier(featuresCol="featuresScaled", labelCol="label", k=1, predictionCol="predicted")
knn_model=classifier.fit(train_set)
transformed: DataFrame = knn_model.transform(test_set)

In [8]:
from evaluator import ArrayClassificationEvaluator
eval = ArrayClassificationEvaluator(labelCol = "label", predictionCol = "predicted")

In [9]:
eval.evaluate(transformed)

{0: 0.23728813559322035, 1: 0.24858757062146894, 2: 0.22033898305084745}